# 1. Preparation

In [1]:
import setproctitle

setproctitle.setproctitle("python")
import os
import time
import sys
import chromadb
import bm25s
import Stemmer
import numpy as np

sys.path.append("../..")

from tqdm import tqdm
from transformers import set_seed
from FlagEmbedding import FlagLLMReranker
from chromadb.api.models.Collection import Collection
from benchmark_generator.context.utils.jsonl import read_jsonl

os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
set_seed(42, deterministic=True)

/raid/p-storage/luthfibalaka_d2b65c57/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
stemmer = Stemmer.Stemmer("english")
reranker = FlagLLMReranker("../../models/bge-reranker")

If you want to use `XLMRobertaLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of XLMRobertaForCausalLM were not initialized from the model checkpoint at ../../models/bge-reranker and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
dataset = "adventure"
if dataset == "chicago":
    narrations = read_jsonl("../../pneuma_summarizer/summaries/narrations/chicago_narrations.jsonl")
    rows = read_jsonl("../../pneuma_summarizer/summaries/rows/chicago.jsonl")
    content_benchmark = read_jsonl("../../data_src/benchmarks/content/pneuma_chicago_10K_questions_annotated.jsonl")
    path = "../../data_src/tables/pneuma_chicago_10K"
elif dataset == "public":
    narrations = read_jsonl("../../pneuma_summarizer/summaries/narrations/public_narrations.jsonl")
    rows = read_jsonl("../../pneuma_summarizer/summaries/rows/public.jsonl")
    content_benchmark = read_jsonl("../../data_src/benchmarks/content/pneuma_public_bi_questions_annotated.jsonl")
    path = "../../data_src/tables/pneuma_public_bi"
elif dataset == "chembl":
    narrations = read_jsonl("../../pneuma_summarizer/summaries/narrations/chembl_narrations.jsonl")
    rows = read_jsonl("../../pneuma_summarizer/summaries/rows/chembl.jsonl")
    contexts = read_jsonl("../../data_src/benchmarks/context/chembl/contexts_chembl.jsonl")
    content_benchmark = read_jsonl("../../data_src/benchmarks/content/pneuma_chembl_10K_questions_annotated.jsonl")
    context_benchmark = read_jsonl("../../data_src/benchmarks/context/chembl/bx_chembl.jsonl")
    path = "../../data_src/tables/pneuma_chembl_10K"
elif dataset == "adventure":
    narrations = read_jsonl("../../pneuma_summarizer/summaries/narrations/adventure_narrations.jsonl")
    rows = read_jsonl("../../pneuma_summarizer/summaries/rows/adventure.jsonl")
    contexts = read_jsonl("../../data_src/benchmarks/context/adventure/contexts_adventure.jsonl")
    content_benchmark = read_jsonl("../../data_src/benchmarks/content/pneuma_adventure_works_questions_annotated.jsonl")
    context_benchmark = read_jsonl("../../data_src/benchmarks/context/adventure/bx_adventure.jsonl")
    path = "../../data_src/tables/pneuma_adventure_works"
elif dataset == "fetaqa":
    narrations = read_jsonl("../../pneuma_summarizer/summaries/narrations/fetaqa_narrations.jsonl")
    rows = read_jsonl("../../pneuma_summarizer/summaries/rows/fetaqa.jsonl")
    content_benchmark = read_jsonl("../../data_src/benchmarks/content/pneuma_fetaqa_questions_annotated.jsonl")
    path = "../../data_src/tables/pneuma_fetaqa"

# 2. Indexing

In [17]:
def indexing_keyword(
    stemmer,
    narration_contents: list[dict[str, str]],
    contexts: list[dict[str, str]] = None,
):
    corpus_json = []
    tables = sorted({content["table"] for content in narration_contents})
    for table in tables:
        cols_descriptions = [
            content["summary"]
            for content in narration_contents
            if content["table"] == table
        ]
        for content_idx, content in enumerate(cols_descriptions):
            corpus_json.append(
                {
                    "text": content,
                    "metadata": {"table": f"{table}_SEP_contents_{content_idx}"},
                }
            )

        if contexts is not None:
            filtered_contexts = [
                context["context"] for context in contexts if context["table"] == table
            ]
            for context_idx, context in enumerate(filtered_contexts):
                corpus_json.append(
                    {
                        "text": context,
                        "metadata": {"table": f"{table}_SEP_{context_idx}"},
                    }
                )

    corpus_text = [doc["text"] for doc in corpus_json]
    corpus_tokens = bm25s.tokenize(
        corpus_text, stopwords="en", stemmer=stemmer, show_progress=False
    )

    retriever = bm25s.BM25(corpus=corpus_json)
    retriever.index(corpus_tokens, show_progress=False)
    return retriever

In [18]:
print(f"Processing {dataset} dataset")
start = time.time()
client = chromadb.PersistentClient(
    f"../indices/index-{dataset}-pneuma-summarizer"
)
collection = client.get_collection("benchmark")
retriever = indexing_keyword(stemmer, rows + narrations, contexts)
end = time.time()
print(f"Indexing time: {end-start} seconds")

Processing adventure dataset
Indexing time: 0.8737316131591797 seconds


# 3. Benchmarking

In [6]:
def process_nodes_bm25(items):
    # Normalize relevance scores and return the nodes in dict format
    results, scores = items
    scores: list[float] = scores[0]
    max_score = max(scores)
    min_score = min(scores)

    processed_nodes: dict[str, tuple[float, str]] = {}
    for i, node in enumerate(results[0]):
        if min_score == max_score:
            score = 1
        else:
            score = (scores[i] - min_score) / (max_score - min_score)
        processed_nodes[node["metadata"]["table"]] = (score, node["text"])
    return processed_nodes

In [7]:
def process_nodes_vec(items):
    # Normalize relevance scores and return the nodes in dict format
    scores: list[float] = [1 - dist for dist in items["distances"][0]]
    max_score = max(scores)
    min_score = min(scores)

    processed_nodes: dict[str, tuple[float, str]] = {}

    for idx in range(len(items["ids"][0])):
        if min_score == max_score:
            score = 1
        else:
            score = (scores[idx] - min_score) / (max_score - min_score)
        processed_nodes[items["ids"][0][idx]] = (score, items["documents"][0][idx])
    return processed_nodes

In [8]:
def hybrid_retriever(
    bm25_res,
    vec_res,
    k: int,
    question: str,
    use_reranker=False,
):
    processed_nodes_bm25 = process_nodes_bm25(bm25_res)
    processed_nodes_vec = process_nodes_vec(vec_res)

    node_ids = set(list(processed_nodes_bm25.keys()) + list(processed_nodes_vec.keys()))
    all_nodes: list[tuple[str, float, str]] = []
    for node_id in sorted(node_ids):
        bm25_score_doc = processed_nodes_bm25.get(node_id, (0.0, None))
        vec_score_doc = processed_nodes_vec.get(node_id, (0.0, None))

        combined_score = 0.5 * bm25_score_doc[0] + 0.5 * vec_score_doc[0]
        if bm25_score_doc[1] is None:
            doc = vec_score_doc[1]
        else:
            doc = bm25_score_doc[1]

        all_nodes.append((node_id, combined_score, doc))

    sorted_nodes = sorted(all_nodes, key=lambda node: (-node[1], node[0]))[:k]
    if use_reranker:
        sorted_nodes = rerank(sorted_nodes, question)
    return sorted_nodes

In [9]:
def rerank(nodes: list[tuple[str, float, str]], question: str):
    # Each node is of the form (name, score, doc)
    names = []
    docs = []
    for node in nodes:
        names.append(node[0])
        docs.append(node[2])
    similarities = reranker.compute_score([(question, doc) for doc in docs], normalize=True)
    reranked_nodes = sorted(zip(names, similarities, docs), key=lambda x: (-x[1], x[0]))
    return reranked_nodes

In [10]:
def get_question_key(benchmark_type: str, use_rephrased_questions: bool):
    if benchmark_type == "content":
        if not use_rephrased_questions:
            question_key = "question_from_sql_1"
        else:
            question_key = "question"
    else:
        if not use_rephrased_questions:
            question_key = "question_bx1"
        else:
            question_key = "question_bx2"
    return question_key

In [11]:
def evaluate_benchmark(
    benchmark: list[dict[str, str]],
    benchmark_type: str,
    k: int,
    collection: Collection,
    retriever,
    stemmer,
    n=3,
    use_reranker=False,
    use_rephrased_questions=False,
):
    start = time.time()
    hitrate_sum = 0
    wrong_questions = []

    if use_reranker:
        increased_k = k * n
    else:
        increased_k = k * n

    question_key = get_question_key(benchmark_type, use_rephrased_questions)

    questions = []
    for data in benchmark:
        questions.append(data[question_key])
    embed_questions = np.loadtxt(
        f"../embeddings/embed-{dataset}-questions-{benchmark_type}-{use_rephrased_questions}.txt"
    )
    embed_questions = [embed.tolist() for embed in embed_questions]

    for idx, datum in enumerate(tqdm(benchmark)):
        answer_tables = datum["answer_tables"]
        question_embedding = embed_questions[idx]

        query_tokens = bm25s.tokenize(
            questions[idx], stemmer=stemmer, show_progress=False
        )
        results, scores = retriever.retrieve(
            query_tokens, k=increased_k, show_progress=False
        )
        bm25_res = (results, scores)

        vec_res = collection.query(
            query_embeddings=[question_embedding], n_results=increased_k
        )

        all_nodes = hybrid_retriever(
            bm25_res, vec_res, increased_k, questions[idx], use_reranker
        )
        before = hitrate_sum
        for table, _, _ in all_nodes[:k]:
            table = table.split("_SEP_")[0]
            if table in answer_tables:
                hitrate_sum += 1
                break
        if before == hitrate_sum:
            wrong_questions.append(idx)
        # Checkpoint
        if idx % 20 == 0:
            print(f"Current Hit Rate Sum at index {idx}: {hitrate_sum}")
            print(
                f"Current wrongly answered questions at index {idx}: {wrong_questions}"
            )

    end = time.time()
    print(f"Hit Rate: {round(hitrate_sum/len(benchmark) * 100, 2)}")
    print(f"Benchmarking Time: {end - start} seconds")
    print(f"Wrongly answered questions: {wrong_questions}")

In [19]:
ks = [1]
ns = [5]
use_reranker = True

In [20]:
for k in ks:
    for n in ns:
        print(f"BC1 with k={k} n={n}")
        evaluate_benchmark(
            content_benchmark,
            "content",
            k,
            collection,
            retriever,
            stemmer,
            n=n,
            use_reranker=use_reranker,
        )
        print("=" * 50)

BC1 with k=1 n=5


  0%|          | 1/1000 [00:00<07:13,  2.31it/s]

Current Hit Rate Sum at index 0: 1
Current wrongly answered questions at index 0: []


100%|██████████| 1/1 [00:00<00:00, 13.54it/s]t/s]


Current Hit Rate Sum at index 20: 17
Current wrongly answered questions at index 20: [5, 9, 14, 15]


...

Current Hit Rate Sum at index 960: 706
Current wrongly answered questions at index 960: [5, 9, 14, 15, 24, 26, 27, 28, 35, 37, 42, 43, 44, 51, 52, 58, 62, 65, 68, 96, 97, 98, 99, 100, 101, 109, 112, 119, 121, 122, 123, 124, 125, 139, 143, 144, 145, 146, 147, 151, 152, 154, 155, 159, 161, 171, 177, 179, 180, 185, 186, 193, 194, 195, 197, 203, 205, 206, 207, 208, 210, 230, 237, 238, 243, 245, 251, 256, 263, 266, 267, 268, 271, 273, 274, 281, 300, 306, 307, 309, 311, 313, 319, 322, 323, 324, 326, 328, 339, 346, 350, 352, 355, 363, 364, 370, 373, 380, 381, 395, 397, 398, 399, 410, 411, 412, 413, 419, 420, 424, 438, 441, 443, 447, 458, 463, 464, 465, 469, 482, 483, 485, 489, 490, 493, 500, 502, 503, 504, 506, 507, 508, 509, 511, 514, 518, 521, 525, 529, 533, 539, 540, 542, 551, 552, 554, 559, 575, 576, 577, 578, 579, 587, 588, 589, 593, 595, 605, 606, 608, 609, 612, 613, 614, 616, 619, 621, 623, 624, 629, 630, 634, 635, 636, 638, 641, 642, 644, 653, 654, 656, 661, 663, 671, 679, 683, 684, 6

 98%|█████████▊| 981/1000 [02:07<00:02,  7.38it/s]

Current Hit Rate Sum at index 980: 724
Current wrongly answered questions at index 980: [5, 9, 14, 15, 24, 26, 27, 28, 35, 37, 42, 43, 44, 51, 52, 58, 62, 65, 68, 96, 97, 98, 99, 100, 101, 109, 112, 119, 121, 122, 123, 124, 125, 139, 143, 144, 145, 146, 147, 151, 152, 154, 155, 159, 161, 171, 177, 179, 180, 185, 186, 193, 194, 195, 197, 203, 205, 206, 207, 208, 210, 230, 237, 238, 243, 245, 251, 256, 263, 266, 267, 268, 271, 273, 274, 281, 300, 306, 307, 309, 311, 313, 319, 322, 323, 324, 326, 328, 339, 346, 350, 352, 355, 363, 364, 370, 373, 380, 381, 395, 397, 398, 399, 410, 411, 412, 413, 419, 420, 424, 438, 441, 443, 447, 458, 463, 464, 465, 469, 482, 483, 485, 489, 490, 493, 500, 502, 503, 504, 506, 507, 508, 509, 511, 514, 518, 521, 525, 529, 533, 539, 540, 542, 551, 552, 554, 559, 575, 576, 577, 578, 579, 587, 588, 589, 593, 595, 605, 606, 608, 609, 612, 613, 614, 616, 619, 621, 623, 624, 629, 630, 634, 635, 636, 638, 641, 642, 644, 653, 654, 656, 661, 663, 671, 679, 683, 684, 6

100%|██████████| 1000/1000 [02:09<00:00,  7.71it/s]

Hit Rate: 73.7
Benchmarking Time: 130.10630321502686 seconds
Wrongly answered questions: [5, 9, 14, 15, 24, 26, 27, 28, 35, 37, 42, 43, 44, 51, 52, 58, 62, 65, 68, 96, 97, 98, 99, 100, 101, 109, 112, 119, 121, 122, 123, 124, 125, 139, 143, 144, 145, 146, 147, 151, 152, 154, 155, 159, 161, 171, 177, 179, 180, 185, 186, 193, 194, 195, 197, 203, 205, 206, 207, 208, 210, 230, 237, 238, 243, 245, 251, 256, 263, 266, 267, 268, 271, 273, 274, 281, 300, 306, 307, 309, 311, 313, 319, 322, 323, 324, 326, 328, 339, 346, 350, 352, 355, 363, 364, 370, 373, 380, 381, 395, 397, 398, 399, 410, 411, 412, 413, 419, 420, 424, 438, 441, 443, 447, 458, 463, 464, 465, 469, 482, 483, 485, 489, 490, 493, 500, 502, 503, 504, 506, 507, 508, 509, 511, 514, 518, 521, 525, 529, 533, 539, 540, 542, 551, 552, 554, 559, 575, 576, 577, 578, 579, 587, 588, 589, 593, 595, 605, 606, 608, 609, 612, 613, 614, 616, 619, 621, 623, 624, 629, 630, 634, 635, 636, 638, 641, 642, 644, 653, 654, 656, 661, 663, 671, 679, 683, 684, 

In [21]:
for k in ks:
    for n in ns:
        print(f"BC2 with k={k} n={n}")
        evaluate_benchmark(
            content_benchmark,
            "content",
            k,
            collection,
            retriever,
            stemmer,
            n=n,
            use_rephrased_questions=True,
            use_reranker=use_reranker,
        )
    print("=" * 50)

BC2 with k=1 n=5


100%|██████████| 1/1 [00:00<00:00, 11.07it/s]


Current Hit Rate Sum at index 0: 1
Current wrongly answered questions at index 0: []


100%|██████████| 1/1 [00:00<00:00, 16.25it/s]t/s]


Current Hit Rate Sum at index 20: 16
Current wrongly answered questions at index 20: [4, 5, 7, 9, 14]


...

Current Hit Rate Sum at index 960: 682
Current wrongly answered questions at index 960: [4, 5, 7, 9, 14, 24, 26, 27, 28, 29, 35, 37, 42, 43, 44, 45, 51, 52, 57, 58, 65, 68, 70, 71, 77, 80, 96, 97, 100, 101, 105, 108, 109, 111, 115, 118, 121, 122, 123, 125, 126, 131, 139, 143, 144, 145, 146, 147, 148, 151, 152, 153, 154, 156, 158, 159, 160, 161, 163, 165, 175, 176, 177, 178, 179, 180, 186, 193, 194, 195, 198, 204, 206, 225, 226, 229, 230, 237, 238, 240, 243, 245, 256, 257, 258, 267, 268, 271, 276, 292, 303, 306, 307, 314, 318, 322, 323, 324, 334, 337, 348, 349, 352, 354, 355, 361, 362, 363, 367, 373, 381, 386, 395, 397, 399, 400, 410, 411, 420, 421, 438, 443, 444, 449, 458, 460, 461, 463, 464, 475, 482, 485, 491, 498, 499, 500, 502, 503, 504, 506, 507, 508, 514, 517, 518, 525, 527, 529, 539, 540, 551, 552, 553, 554, 555, 564, 576, 577, 578, 585, 587, 588, 589, 592, 593, 594, 598, 599, 600, 605, 609, 612, 613, 616, 617, 619, 620, 621, 629, 630, 632, 633, 635, 641, 642, 644, 647, 654, 656

 98%|█████████▊| 981/1000 [02:07<00:02,  7.61it/s]

Current Hit Rate Sum at index 980: 697
Current wrongly answered questions at index 980: [4, 5, 7, 9, 14, 24, 26, 27, 28, 29, 35, 37, 42, 43, 44, 45, 51, 52, 57, 58, 65, 68, 70, 71, 77, 80, 96, 97, 100, 101, 105, 108, 109, 111, 115, 118, 121, 122, 123, 125, 126, 131, 139, 143, 144, 145, 146, 147, 148, 151, 152, 153, 154, 156, 158, 159, 160, 161, 163, 165, 175, 176, 177, 178, 179, 180, 186, 193, 194, 195, 198, 204, 206, 225, 226, 229, 230, 237, 238, 240, 243, 245, 256, 257, 258, 267, 268, 271, 276, 292, 303, 306, 307, 314, 318, 322, 323, 324, 334, 337, 348, 349, 352, 354, 355, 361, 362, 363, 367, 373, 381, 386, 395, 397, 399, 400, 410, 411, 420, 421, 438, 443, 444, 449, 458, 460, 461, 463, 464, 475, 482, 485, 491, 498, 499, 500, 502, 503, 504, 506, 507, 508, 514, 517, 518, 525, 527, 529, 539, 540, 551, 552, 553, 554, 555, 564, 576, 577, 578, 585, 587, 588, 589, 592, 593, 594, 598, 599, 600, 605, 609, 612, 613, 616, 617, 619, 620, 621, 629, 630, 632, 633, 635, 641, 642, 644, 647, 654, 656

100%|██████████| 1000/1000 [02:09<00:00,  7.74it/s]

Hit Rate: 70.4
Benchmarking Time: 129.61282396316528 seconds
Wrongly answered questions: [4, 5, 7, 9, 14, 24, 26, 27, 28, 29, 35, 37, 42, 43, 44, 45, 51, 52, 57, 58, 65, 68, 70, 71, 77, 80, 96, 97, 100, 101, 105, 108, 109, 111, 115, 118, 121, 122, 123, 125, 126, 131, 139, 143, 144, 145, 146, 147, 148, 151, 152, 153, 154, 156, 158, 159, 160, 161, 163, 165, 175, 176, 177, 178, 179, 180, 186, 193, 194, 195, 198, 204, 206, 225, 226, 229, 230, 237, 238, 240, 243, 245, 256, 257, 258, 267, 268, 271, 276, 292, 303, 306, 307, 314, 318, 322, 323, 324, 334, 337, 348, 349, 352, 354, 355, 361, 362, 363, 367, 373, 381, 386, 395, 397, 399, 400, 410, 411, 420, 421, 438, 443, 444, 449, 458, 460, 461, 463, 464, 475, 482, 485, 491, 498, 499, 500, 502, 503, 504, 506, 507, 508, 514, 517, 518, 525, 527, 529, 539, 540, 551, 552, 553, 554, 555, 564, 576, 577, 578, 585, 587, 588, 589, 592, 593, 594, 598, 599, 600, 605, 609, 612, 613, 616, 617, 619, 620, 621, 629, 630, 632, 633, 635, 641, 642, 644, 647, 654, 65